In [1]:
"""
Extract gridMET variables for daily burn perimeters, western U.S.
Google Earth Engine (GEE) Python API
Author: maxwell.cook@colorado.edu
"""

import ee
import geemap

ee.Authenticate()
ee.Initialize(project='jfsp-aspen')
print("GEE Authenticated !")

GEE Authenticated !


In [ ]:
# Function to calculate gridmet for daily perimeters
def gridmet_summary(fire, reducer, buffer=None):
    """ Calculates the gridMET variables for a given daily perimeter by daily ID """
    # Set a function to calculate the variables for each daily perimeter
    def get_daily_gridmet(perim):
        # Buffer if requested
        if buffer is not None:
            geom = perim.geometry().bounds().buffer(buffer)
        else:
            geom = perim.geometry().bounds().buffer(buffer)
        did = perim.get('did') # get the daily burn ID
        # Get the burn date for extracting gridMET variables
        burn_date = ee.Date(feature.get('date')) 
        gridmet_ = gridmet.filterDate(burn_date, burn_date.advance(1, 'day')).select(vars)
        # Set up the feature reduction
        stats = gridmet_.reduceRegion(
            reducer=reduction,
            geometry=geom,
            scale=4000,  # Match gridMET resolution
            bestEffort=True)
        return perim.set(stats)
    results = fire.map(get_daily_gridmet)
    return results

print("Functions loaded !")

In [ ]:
# Load the gridMET datasets
gridmet = ee.ImageCollection('IDAHO_EPSCOR/GRIDMET')

# Define the parameters
vars = ['vs','vpd','erc','bi','fm1000'] # gridmet variables
reducer = ee.Reducer.percentile([90,95,99]) # reducer